In [ ]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import glob
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn import preprocessing
from keras.utils import to_categorical
from keras import metrics
from collections import Counter
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import tensorflow as tf
from keras.layers import LSTM
from sklearn.model_selection import StratifiedKFold, cross_val_score
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasClassifier
from keras.utils import to_categorical
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
# load all csv files
path = '/content/drive/MyDrive/Data/EDA/1_5'  # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

# separate into input (X) and output (y) variables
X = df[['mean', 'std_dev', 'range']].values
y = df['Label'].values

# encode class values as integers
y = np_utils.to_categorical(y)

# split into 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Revert one-hot-encoded y_train back to class labels
y_train_labels = np.argmax(y_train, axis=1)
# reshape from [samples, features] into [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model.add(Dense(32, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=1, validation_split=0.3, callbacks=[early_stopping])

# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

# make class predictions with the model
probabilities = model.predict(X_test)
predictions = np.argmax(probabilities, axis=-1)

# summarize the first 5 cases
for i in range(5):
    print('%s => %d (expected %d)' % (X_test[i].tolist(), predictions[i], np.argmax(y_test[i])))

# confusion matrix
print(confusion_matrix(np.argmax(y_test, axis=1), predictions))

# classification report
print(classification_report(np.argmax(y_test, axis=1), predictions))


Epoch 1/150
31/31 [==============================] - 2s 16ms/step - loss: 1.4114 - accuracy: 0.7045 - val_loss: 1.2931 - val_accuracy: 0.6992
Epoch 2/150
31/31 [==============================] - 0s 5ms/step - loss: 1.1783 - accuracy: 0.7338 - val_loss: 1.0736 - val_accuracy: 0.7143
Epoch 3/150
31/31 [==============================] - 0s 4ms/step - loss: 0.9511 - accuracy: 0.7760 - val_loss: 0.8611 - val_accuracy: 0.7368
Epoch 4/150
31/31 [==============================] - 0s 4ms/step - loss: 0.7525 - accuracy: 0.8084 - val_loss: 0.7162 - val_accuracy: 0.7669
Epoch 5/150
31/31 [==============================] - 0s 5ms/step - loss: 0.6264 - accuracy: 0.8279 - val_loss: 0.6263 - val_accuracy: 0.7820
Epoch 6/150
31/31 [==============================] - 0s 6ms/step - loss: 0.5479 - accuracy: 0.8247 - val_loss: 0.5655 - val_accuracy: 0.7895
Epoch 7/150
31/31 [==============================] - 0s 4ms/step - loss: 0.4988 - accuracy: 0.8377 - val_loss: 0.5242 - val_accuracy: 0.8045
Epoch 8/150
